# Combine all data sets into one

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
DATA_PATH = r'C:\Users\connor\PycharmProjects\trading\data\base'
ANALYTICS_PATH = r'C:\Users\connor\PycharmProjects\trading\data\analytics'

TABLE_PATHS_AND_GRAINS = [[r'\ema\voo_ema.csv', 'daily'], [r'\fred_data\CPALTT01USM657N.csv', 'monthly'], [r'\fred_data\DFF.csv', 'daily'], [r'\fred_data\EXPINF10YR.csv', 'monthly'], [r'\fred_data\GDPC1.csv', 'quarterly'], [r'\fred_data\RSXFS.csv', 'monthly'], [r'\fred_data\T10YFF.csv', 'daily'], [r'\fred_data\UNRATE.csv', 'monthly'], [r'\macdext\voo_macdext.csv', 'daily'], [r'\obv\voo_obv.csv', 'daily'], [r'\voo_intraday\full_voo.csv', 'minute']]

# T10YFF has . instead of nulls. Annoying
df_dict = {}

for table, grain in TABLE_PATHS_AND_GRAINS:
    # create file paths for each table
    table = table
    file_path = DATA_PATH + table
    table_name = table.rsplit('\\', 1)[1]
    
    # store dfs in dictionary
    df_dict[table_name] = [pd.read_csv(file_path), grain]

In [3]:
# load calendar
STAGE_CALENDAR_PATH = r'C:\Users\connor\PycharmProjects\trading\data\stage\stage_calendar\stage_calendar.csv'
df_calendar = pd.read_csv(STAGE_CALENDAR_PATH)

# have to cast join key. Later, go back and 
df_calendar.DateShortDescription = df_calendar.DateShortDescription.astype('datetime64[ns]')

In [4]:
# join calendar to voo daily

# cast date as datetime
df_dict['full_voo.csv'][0] = df_dict['full_voo.csv'][0].astype({'date': 'datetime64[ns]', 'open': float, 'high': float, 'low': float, 'close': float, 'volume': int})

# Add column to VOO that is a date_trunc with no timestamp to allow other datasets to join to it
df_dict['full_voo.csv'][0]['join_date'] = df_dict['full_voo.csv'][0]['date'].dt.date

# recast (again) to make join_date a datetime object
df_dict['full_voo.csv'][0] = df_dict['full_voo.csv'][0].astype({'date': 'datetime64[ns]', 'open': float, 'high': float, 'low': float, 'close': float, 'volume': int, 'join_date': 'datetime64[ns]'})

df_final = df_dict['full_voo.csv'][0].copy()

# join calendar to voo
df_final = pd.merge(left=df_final, right=df_calendar, how='left', left_on='join_date', right_on='DateShortDescription')

# confirm join was successful and there are no nulls
df_final.info()
df_final.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1428573 entries, 0 to 1428572
Data columns (total 37 columns):
 #   Column                         Non-Null Count    Dtype         
---  ------                         --------------    -----         
 0   date                           1428573 non-null  datetime64[ns]
 1   open                           1428573 non-null  float64       
 2   high                           1428573 non-null  float64       
 3   low                            1428573 non-null  float64       
 4   close                          1428573 non-null  float64       
 5   volume                         1428573 non-null  int32         
 6   join_date                      1428573 non-null  datetime64[ns]
 7   Id                             1428573 non-null  int64         
 8   Date                           1428573 non-null  object        
 9   DateLongDescription            1428573 non-null  object        
 10  DateShortDescription           1428573 non-null  datet

,date,open,high,low,close,volume,join_date,Id,Date,DateLongDescription,...,CalendarQuarterEndDateId,CalendarQuarterStartDate,CalendarNumberOfDaysInQuarter,CalendarDayInQuarter,CalendarYear,CalendarYearEndDateId,CalendarYearStartDate,CalendarNumberOfDaysInYear,month_join_key,year_join_key
0,2010-09-09 09:30:00,79.250,79.351,78.323,78.506,2600,2010-09-09,100809,9/9/2010,"Thursday, September 9, 2010",...,100830,2010-07-01,92,71,2010,101131,1/1/2010,365,2010-09-01,2010-01-01
1,2010-09-09 09:31:00,78.399,78.499,78.292,78.429,100,2010-09-09,100809,9/9/2010,"Thursday, September 9, 2010",...,100830,2010-07-01,92,71,2010,101131,1/1/2010,365,2010-09-01,2010-01-01
2,2010-09-09 09:33:00,79.080,79.181,78.971,79.109,200,2010-09-09,100809,9/9/2010,"Thursday, September 9, 2010",...,100830,2010-07-01,92,71,2010,101131,1/1/2010,365,2010-09-01,2010-01-01
3,2010-09-09 09:36:00,79.064,79.165,78.956,79.094,100,2010-09-09,100809,9/9/2010,"Thursday, September 9, 2010",...,100830,2010-07-01,92,71,2010,101131,1/1/2010,365,2010-09-01,2010-01-01
4,2010-09-09 09:44:00,78.569,78.716,78.462,78.645,750,2010-09-09,100809,9/9/2010,"Thursday, September 9, 2010",...,100830,2010-07-01,92,71,2010,101131,1/1/2010,365,2010-09-01,2010-01-01


In [5]:
for table_name, df_current in df_dict.items():
    if table_name[0] != 'full_voo.csv':
        grain = df_current[1]
        df_temp = df_current[0]
        
        # have to have dtype be the same for the join to work. Having the same names isn't enough
        df_final['join_date'] = df_final.join_date.astype(str)
        df_temp['date'] = df_temp.date.astype(str)

        if grain == 'daily':
            df_final = pd.merge(left=df_final, right=df_temp.rename(columns={'date': 'join_date'}), on='join_date', how='left')
            
        if grain == 'monthly':
            df_final = pd.merge(left=df_final, right=df_temp.rename(columns={'date': 'month_join_key'}), on='month_join_key', how='left')
            
        if grain == 'quarterly':
            df_final = pd.merge(left=df_final, right=df_temp.rename(columns={'date': 'CalendarQuarterStartDate'}), on='CalendarQuarterStartDate', how='left')
            
        if grain == 'annually':
            df_final = pd.merge(left=df_final, right=df_temp.rename(columns={'date': 'year_join_key'}), on='year_join_key', how='left')

In [6]:
# want to predict whether target was positive or negative. Change target to be a True False value from a float
df_final['target'] = df_final['close'].shift(periods=-1)
df_final.dropna(inplace=True)

def target_to_binary(df):
    if df['target'] > df['close']:
        return 1
    else:
        return 0

df_final['target'] = df_final.apply(target_to_binary, axis=1)

In [7]:
# T10YFF has . instead of null. 
df_final['T10YFF'] = df_final['T10YFF'].replace('.', np.NaN)
df_final['T10YFF'] = df_final['T10YFF'].fillna(method='ffill')
df_final['T10YFF'] = df_final['T10YFF'].astype(float)

C:\Users\connor\AppData\Local\Temp\ipykernel_7944\2405079809.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_final['T10YFF'] = df_final['T10YFF'].fillna(method='ffill')


In [8]:
df_final.head()

,date,open,high,low,close,volume,join_date,Id,Date,DateLongDescription,...,EXPINF10YR,GDPC1,RSXFS,T10YFF,UNRATE,macd,macd_signal,macd_hist,daily_obv,target
2595,2010-10-26 09:28:00,83.935,84.042,83.820,83.967,232,2010-10-26,100926,10/26/2010,"Tuesday, October 26, 2010",...,1.574237,16960.864,323990.0,2.48,9.4,1.1427,1.1947,-0.0521,-148100.0,0
2596,2010-10-26 09:30:00,83.888,83.995,83.743,83.889,150,2010-10-26,100926,10/26/2010,"Tuesday, October 26, 2010",...,1.574237,16960.864,323990.0,2.48,9.4,1.1427,1.1947,-0.0521,-148100.0,1
2597,2010-10-26 09:39:00,83.888,83.995,83.774,83.920,1600,2010-10-26,100926,10/26/2010,"Tuesday, October 26, 2010",...,1.574237,16960.864,323990.0,2.48,9.4,1.1427,1.1947,-0.0521,-148100.0,1
2598,2010-10-26 09:41:00,83.904,84.011,83.776,83.923,100,2010-10-26,100926,10/26/2010,"Tuesday, October 26, 2010",...,1.574237,16960.864,323990.0,2.48,9.4,1.1427,1.1947,-0.0521,-148100.0,1
2599,2010-10-26 09:51:00,83.966,84.073,83.851,83.998,698,2010-10-26,100926,10/26/2010,"Tuesday, October 26, 2010",...,1.574237,16960.864,323990.0,2.48,9.4,1.1427,1.1947,-0.0521,-148100.0,1


In [9]:
df_final.head() 
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1425977 entries, 2595 to 1428571
Data columns (total 50 columns):
 #   Column                         Non-Null Count    Dtype         
---  ------                         --------------    -----         
 0   date                           1425977 non-null  datetime64[ns]
 1   open                           1425977 non-null  float64       
 2   high                           1425977 non-null  float64       
 3   low                            1425977 non-null  float64       
 4   close                          1425977 non-null  float64       
 5   volume                         1425977 non-null  int32         
 6   join_date                      1425977 non-null  object        
 7   Id                             1425977 non-null  int64         
 8   Date                           1425977 non-null  object        
 9   DateLongDescription            1425977 non-null  object        
 10  DateShortDescription           1425977 non-null  datetim

In [10]:
df_final.to_csv(os.path.join(ANALYTICS_PATH, 'analytics_voo.csv'), index=False)